# Quantum Teleportation

In [1]:
from sympy.physics.quantum.qapply import qapply
from sympy.physics.quantum.qubit import Qubit, matrix_to_qubit
from sympy import Symbol, sqrt, simplify
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.gate import CNOT, XGate, ZGate, HadamardGate
from sympy.physics.quantum.represent import represent

The problem:
Teleport a quantum state

The Solution:
```
             SENDING   |  RECEIVING
               --------|--X-Z
     |PHI+>    ----X---|--o-|
     a|0>+b|1> ----o-H-|----o
```

## Sending side

Prepare `PHI+` state

In [2]:
phi_p = TensorProduct(Qubit(0), Qubit(0))
phi_p = matrix_to_qubit(represent(phi_p))
phi_p = qapply(HadamardGate(1) * phi_p)
phi_p = qapply(CNOT(1, 0) * phi_p)
phi_p = matrix_to_qubit(represent(phi_p))
print(f"|PHI+>: {phi_p}")
assert (simplify(phi_p - 1 / sqrt(2) * (Qubit('00') + Qubit('11'))) == 0)

|PHI+>: sqrt(2)*|00>/2 + sqrt(2)*|11>/2


Prepare qubit to teleport

In [3]:
alpha = Symbol('alpha', real=True)
beta = Symbol('beta', real=True)
q = alpha * Qubit(0) + beta * Qubit(1)
print(f"State to teleport: |q>={q}")


State to teleport: |q>=alpha*|0> + beta*|1>


full input state is 

In [4]:
full_state = TensorProduct(q, phi_p)
full_state = matrix_to_qubit(represent(full_state))
print(f"|q>|PHI+>={full_state}")


|q>|PHI+>=sqrt(2)*alpha*|000>/2 + sqrt(2)*alpha*|011>/2 + sqrt(2)*beta*|100>/2 + sqrt(2)*beta*|111>/2


In [5]:
# apply CNOT
print("Apply CNOT")
c = CNOT(2, 1)  # that qubits are indexed from right to left
full_state = qapply(c * full_state)
print(f"CNOT_2_1*|q>|PHI+>={full_state}")


Apply CNOT
CNOT_2_1*|q>|PHI+>=sqrt(2)*alpha*|000>/2 + sqrt(2)*alpha*|011>/2 + sqrt(2)*beta*|101>/2 + sqrt(2)*beta*|110>/2


In [6]:
# apply Hadamard
print("Apply Hadamard")
full_state = qapply(HadamardGate(2) * full_state)
print(f"H(2)*CNOT_2_1*|q>|PHI+>={full_state}")


Apply Hadamard
H(2)*CNOT_2_1*|q>|PHI+>=alpha*|000>/2 + alpha*|011>/2 + alpha*|100>/2 + alpha*|111>/2 + beta*|001>/2 + beta*|010>/2 - beta*|101>/2 - beta*|110>/2


In [7]:
# factoring the first 2 qubits:
Q_00 = alpha * Qubit(0) + beta * Qubit(1)
Q_01 = alpha * Qubit(1) + beta * Qubit(0)
Q_10 = alpha * Qubit(0) - beta * Qubit(1)
Q_11 = alpha * Qubit(1) - beta * Qubit(0)

a = (TensorProduct(Qubit('00'), Q_00) +
     TensorProduct(Qubit('01'), Q_01) +
     TensorProduct(Qubit('10'), Q_10) +
     TensorProduct(Qubit('11'), Q_11)) / 2
a = matrix_to_qubit(represent(a))
assert a == full_state

In [8]:
print("\nQ_00 -> nothing to do")
r = Q_00
print(f"({Q_00})={r}")
assert r == q


Q_00 -> nothing to do
(alpha*|0> + beta*|1>)=alpha*|0> + beta*|1>


In [9]:
print("\nQ_01 -> Apply XGate")
r = qapply(XGate(0) * Q_01)
print(f"X*({Q_01})={r}")
assert r == q


Q_01 -> Apply XGate
X*(alpha*|1> + beta*|0>)=alpha*|0> + beta*|1>


In [10]:
print("\nQ_10 -> Apply ZGate")
r = qapply(ZGate(0) * Q_10)
print(f"Z*({Q_10})={r}")
assert r == q


Q_10 -> Apply ZGate
Z*(alpha*|0> - beta*|1>)=alpha*|0> + beta*|1>


In [11]:
print("\nQ_11 -> Apply XGate, ZGate")
r = qapply(XGate(0) * Q_11)
r = qapply(ZGate(0) * r)
print(f"X*Z*({Q_11})={r}")
assert r == q


Q_11 -> Apply XGate, ZGate
X*Z*(alpha*|1> - beta*|0>)=alpha*|0> + beta*|1>


## Examples

In [12]:
# Introduction to classical and quantum computing
# 6.5.3 exercise 6.16
def test_0():
    psi_p = 1 / sqrt(2) * (Qubit('01') + Qubit('10'))
    print("\n|PSI+>: {psi}".format(psi=psi_p))

    # qubit to teleport
    alpha = Symbol('alpha', real=True)
    beta = Symbol('beta', real=True)
    q = alpha * Qubit(0) + beta * Qubit(1)
    print("State to teleport: |q>={q}".format(q=q))

    # full state
    full_state = TensorProduct(q, psi_p)
    full_state = matrix_to_qubit(represent(full_state))
    print("|q>|PHI+>={full}".format(full=full_state))

    # apply CNOT
    print("Apply CNOT")
    c = CNOT(2, 1)  # that qubits are indexed from right to left
    full_state = qapply(c * full_state)
    print("CNOT_2_1*|q>|PHI+>={full}".format(full=full_state))

    # apply Hadamard
    print("Apply Hadamard")
    full_state = qapply(HadamardGate(2) * full_state)
    print("H(2)*CNOT_2_1*|q>|PHI+>={full}".format(full=full_state))

    # factoring the first 2 qubits:
    Q_00 = alpha * Qubit(1) + beta * Qubit(0)
    Q_01 = alpha * Qubit(0) + beta * Qubit(1)
    Q_10 = alpha * Qubit(1) - beta * Qubit(0)
    Q_11 = alpha * Qubit(0) - beta * Qubit(1)

    a = (TensorProduct(Qubit('00'), Q_00) +
         TensorProduct(Qubit('01'), Q_01) +
         TensorProduct(Qubit('10'), Q_10) +
         TensorProduct(Qubit('11'), Q_11)) / 2
    a = matrix_to_qubit(represent(a))
    assert a == full_state

    print("\nQ_00 -> Apply XGate")
    r = qapply(XGate(0) * Q_00)
    print("X*{q}=({r})".format(q=Q_00, r=r))
    assert r == q

    print("\nQ_01 -> nothing to do")
    r = Q_01
    print("{q}=({r})".format(q=Q_01, r=r))
    assert r == q

    print("\nQ_10 -> Apply ZGate ZGate")
    r = qapply(XGate(0) * Q_10)
    r = qapply(ZGate(0) * r)
    print("X*Z*({q})={r}".format(q=Q_10, r=r))
    assert r == q

    print("\nQ_11 -> Apply ZGate")
    r = qapply(ZGate(0) * Q_11)
    print("Z*({q})={r}".format(q=Q_11, r=r))
    assert r == q


test_0()


|PSI+>: sqrt(2)*(|01> + |10>)/2
State to teleport: |q>=alpha*|0> + beta*|1>
|q>|PHI+>=sqrt(2)*alpha*|001>/2 + sqrt(2)*alpha*|010>/2 + sqrt(2)*beta*|101>/2 + sqrt(2)*beta*|110>/2
Apply CNOT
CNOT_2_1*|q>|PHI+>=sqrt(2)*alpha*|001>/2 + sqrt(2)*alpha*|010>/2 + sqrt(2)*beta*|100>/2 + sqrt(2)*beta*|111>/2
Apply Hadamard
H(2)*CNOT_2_1*|q>|PHI+>=alpha*|001>/2 + alpha*|010>/2 + alpha*|101>/2 + alpha*|110>/2 + beta*|000>/2 + beta*|011>/2 - beta*|100>/2 - beta*|111>/2

Q_00 -> Apply XGate
X*alpha*|1> + beta*|0>=(alpha*|0> + beta*|1>)

Q_01 -> nothing to do
alpha*|0> + beta*|1>=(alpha*|0> + beta*|1>)

Q_10 -> Apply ZGate ZGate
X*Z*(alpha*|1> - beta*|0>)=alpha*|0> + beta*|1>

Q_11 -> Apply ZGate
Z*(alpha*|0> - beta*|1>)=alpha*|0> + beta*|1>


In [13]:
def test_1():
    GHZ = 1 / sqrt(2) * (Qubit('000') + Qubit('111'))
    print("\n|GHZ+>: {ghz}".format(ghz=GHZ))

    # qubit to teleport
    alpha = Symbol('alpha', real=True)
    beta = Symbol('beta', real=True)
    psi = alpha * Qubit(0) + beta * Qubit(1)
    print("State to teleport: |psi>={psi}".format(psi=psi))

    # full state
    full_state = TensorProduct(psi, GHZ)
    full_state = matrix_to_qubit(represent(full_state))
    print("|psi>|GHZ>={full}".format(full=full_state))

    # apply CNOT_21
    print("Apply CNOT_21")
    c = CNOT(2, 1)  # that qubits are indexed from right to left
    full_state = qapply(c * full_state)
    print("CNOT_2_1*|psi>|GHZ>={full}".format(full=full_state))

    # apply CNOT_32
    print("Apply CNOT_32")
    c = CNOT(3, 2)  # that qubits are indexed from right to left
    full_state = qapply(c * full_state)
    print("CNOT_3_2*CNOT_2_1*|psi>|GHZ>={full}".format(full=full_state))

    # apply Hadamard
    print("Apply Hadamard")
    full_state = qapply(HadamardGate(3) * full_state)
    print("H(3)*CNOT_3_2*CNOT_2_1*|psi>|GHZ>={full}".format(full=full_state))

    # factoring the first 2 qubits:
    Q_000 = alpha * Qubit(0) + beta * Qubit(1)
    Q_010 = alpha * Qubit(1) + beta * Qubit(0)
    Q_100 = alpha * Qubit(0) - beta * Qubit(1)
    Q_110 = alpha * Qubit(1) - beta * Qubit(0)

    a = (TensorProduct(Qubit('000'), Q_000) +
         TensorProduct(Qubit('010'), Q_010) +
         TensorProduct(Qubit('100'), Q_100) +
         TensorProduct(Qubit('110'), Q_110)) / 2
    a = matrix_to_qubit(represent(a))
    assert a == full_state

    print("Q_000 -> nothing to do")
    r = Q_000
    print("{q}={r}".format(q=Q_000, r=r))
    assert r == psi

    print("\nQ_010 -> Apply XGate")
    r = qapply(XGate(0) * Q_010)
    print("X*({q})=({r})".format(q=Q_010, r=r))
    assert r == psi

    print("\nQ_100 -> Apply ZGate")
    r = qapply(ZGate(0) * Q_100)
    print("Z*({q})=({r})".format(q=Q_100, r=r))

    print("\nQ_110 -> Apply XGate, ZGate")
    r = qapply(XGate(0) * Q_110)
    r = qapply(ZGate(0) * r)
    print("X*Z*({q})=({r})".format(q=Q_110, r=r))
    assert r == psi


test_1()


|GHZ+>: sqrt(2)*(|000> + |111>)/2
State to teleport: |psi>=alpha*|0> + beta*|1>
|psi>|GHZ>=sqrt(2)*alpha*|0000>/2 + sqrt(2)*alpha*|0111>/2 + sqrt(2)*beta*|1000>/2 + sqrt(2)*beta*|1111>/2
Apply CNOT_21
CNOT_2_1*|psi>|GHZ>=sqrt(2)*alpha*|0000>/2 + sqrt(2)*alpha*|0101>/2 + sqrt(2)*beta*|1000>/2 + sqrt(2)*beta*|1101>/2
Apply CNOT_32
CNOT_3_2*CNOT_2_1*|psi>|GHZ>=sqrt(2)*alpha*|0000>/2 + sqrt(2)*alpha*|0101>/2 + sqrt(2)*beta*|1001>/2 + sqrt(2)*beta*|1100>/2
Apply Hadamard
H(3)*CNOT_3_2*CNOT_2_1*|psi>|GHZ>=alpha*|0000>/2 + alpha*|0101>/2 + alpha*|1000>/2 + alpha*|1101>/2 + beta*|0001>/2 + beta*|0100>/2 - beta*|1001>/2 - beta*|1100>/2
Q_000 -> nothing to do
alpha*|0> + beta*|1>=alpha*|0> + beta*|1>

Q_010 -> Apply XGate
X*(alpha*|1> + beta*|0>)=(alpha*|0> + beta*|1>)

Q_100 -> Apply ZGate
Z*(alpha*|0> - beta*|1>)=(alpha*|0> + beta*|1>)

Q_110 -> Apply XGate, ZGate
X*Z*(alpha*|1> - beta*|0>)=(alpha*|0> + beta*|1>)


In [ ]:
def test_2():
    GHZ = 1 / sqrt(2) * (Qubit('000') + Qubit('111'))
    print("\n|GHZ+>: {ghz}".format(ghz=GHZ))

    # qubit to teleport
    alpha = Symbol('alpha', real=True)
    beta = Symbol('beta', real=True)
    psi = alpha * Qubit(0) + beta * Qubit(1)
    print("State to teleport: |psi>={psi}".format(psi=psi))

    # full state
    full_state = TensorProduct(psi, GHZ)
    full_state = matrix_to_qubit(represent(full_state))
    print("|psi>|GHZ>={full}".format(full=full_state))

    # apply CNOT_32
    print("Apply CNOT_32")
    c = CNOT(3, 2)  # that qubits are indexed from right to left
    full_state = qapply(c * full_state)
    print("CNOT_3_2*|psi>|GHZ>={full}".format(full=full_state))

    # apply Hadamard
    print("Apply Hadamard")
    full_state = qapply(HadamardGate(3) * full_state)
    print("H(3)*CNOT_3_2*|psi>|GHZ>={full}".format(full=full_state))

    # factoring the first 2 qubits:
    Q_00 = alpha * Qubit('00') + beta * Qubit('11')
    Q_01 = alpha * Qubit('11') + beta * Qubit('00')
    Q_10 = alpha * Qubit('00') - beta * Qubit('11')
    Q_11 = alpha * Qubit('11') - beta * Qubit('00')

    a = (TensorProduct(Qubit('00'), Q_00) +
         TensorProduct(Qubit('01'), Q_01) +
         TensorProduct(Qubit('10'), Q_10) +
         TensorProduct(Qubit('11'), Q_11)) / 2
    a = matrix_to_qubit(represent(a))
    assert a == full_state

    print("\nApply Hadamard to Q_00")
    r = qapply(HadamardGate(1) * Q_00)
    print("Q_00={r}".format(r=r))
    # factoring the first qubit:
    Q_00_a = alpha * Qubit('0') + beta * Qubit('1')
    Q_00_b = alpha * Qubit('0') - beta * Qubit('1')
    c = 1 / sqrt(2) * (TensorProduct(Qubit('0'), Q_00_a)
                       + TensorProduct(Qubit('1'), Q_00_b))
    c = matrix_to_qubit(represent(c))
    assert c == r

    print("\nApply Hadamard Q_01")
    r = qapply(HadamardGate(1) * Q_01)
    print("Q_01={r}".format(r=r))
    # factoring the first qubit:
    Q_01_a = beta * Qubit('0') + alpha * Qubit('1')
    Q_01_b = beta * Qubit('0') - alpha * Qubit('1')
    c = 1 / sqrt(2) * (TensorProduct(Qubit('0'), Q_01_a)
                       + TensorProduct(Qubit('1'), Q_01_b))
    c = matrix_to_qubit(represent(c))
    assert c == r

    print("\nApply Hadamard Q_10")
    r = qapply(HadamardGate(1) * Q_10)
    print("Q_10={r}".format(r=r))
    # factoring the first qubit:
    Q_10_a = alpha * Qubit('0') - beta * Qubit('1')
    Q_10_b = alpha * Qubit('0') + beta * Qubit('1')
    c = 1 / sqrt(2) * (TensorProduct(Qubit('0'), Q_10_a)
                       + TensorProduct(Qubit('1'), Q_10_b))
    c = matrix_to_qubit(represent(c))
    print("c={c}".format(c=c))
    assert c == r

    print("\nApply Hadamard Q_11")
    r = qapply(HadamardGate(1) * Q_11)
    print("Q_11={r}".format(r=r))
    # factoring the first qubit:
    Q_11_a = (alpha * Qubit('1') - beta * Qubit('0'))
    Q_11_b = (-alpha * Qubit('1') - beta * Qubit('0'))
    c = 1 / sqrt(2) * (TensorProduct(Qubit('0'), Q_11_a)
                       + TensorProduct(Qubit('1'), Q_11_b))
    c = matrix_to_qubit(represent(c))
    assert c == r

    print("\nQ_000  nothing to do")
    r = Q_00_a
    print("{q}={r}".format(q=Q_00_a, r=r))
    assert r == psi

    print("\nQ_001 -> Apply ZGate")
    r = qapply(ZGate(0) * Q_00_b)
    print("Z*({q})={r}".format(q=Q_00_b, r=r))
    assert r == psi

    print("\nQ_010 -> Apply XGate")
    r = qapply(XGate(0) * Q_01_a)
    print("X*({q})={r}".format(q=Q_01_a, r=r))
    assert r == psi

    print("\nQ_011 -> Apply ZGate XGate")
    r = qapply(ZGate(0) * Q_01_b)
    r = qapply(XGate(0) * r)
    print("Z*X*({q})={r}".format(q=Q_01_b, r=r))
    assert r == psi

    print("\nQ_100 -> Apply ZGate")
    r = qapply(ZGate(0) * Q_10_a)
    print("Z*({q})={r}".format(q=Q_10_a, r=r))
    assert r == psi

    print("\nQ_101 -> Nothing")
    r = Q_10_b
    print("{q}={r}".format(q=Q_10_b, r=r))
    assert r == psi

    print("\nQ_110 -> Apply XGate, ZGate")
    r = qapply(XGate(0) * Q_11_a)
    r = qapply(ZGate(0) * r)
    print("X*Z*({q})={r}".format(q=Q_11_a, r=r))
    assert r == psi

    print("\nQ_111 -> ZGate, XGate, ZGate")
    r = qapply(ZGate(0) * Q_11_b)
    r = qapply(XGate(0) * r)
    r = qapply(ZGate(0) * r)
    print("Z*X*Z*({q})={r}".format(q=Q_11_b, r=r))
    assert r == psi


test_2()


|GHZ+>: sqrt(2)*(|000> + |111>)/2
State to teleport: |psi>=alpha*|0> + beta*|1>
|psi>|GHZ>=sqrt(2)*alpha*|0000>/2 + sqrt(2)*alpha*|0111>/2 + sqrt(2)*beta*|1000>/2 + sqrt(2)*beta*|1111>/2
Apply CNOT_32
CNOT_3_2*|psi>|GHZ>=sqrt(2)*alpha*|0000>/2 + sqrt(2)*alpha*|0111>/2 + sqrt(2)*beta*|1011>/2 + sqrt(2)*beta*|1100>/2
Apply Hadamard
H(3)*CNOT_3_2*|psi>|GHZ>=alpha*|0000>/2 + alpha*|0111>/2 + alpha*|1000>/2 + alpha*|1111>/2 + beta*|0011>/2 + beta*|0100>/2 - beta*|1011>/2 - beta*|1100>/2

Apply Hadamard to Q_00
Q_00=sqrt(2)*alpha*|00>/2 + sqrt(2)*alpha*|10>/2 + sqrt(2)*beta*|01>/2 - sqrt(2)*beta*|11>/2

Apply Hadamard Q_01
Q_01=sqrt(2)*alpha*|01>/2 - sqrt(2)*alpha*|11>/2 + sqrt(2)*beta*|00>/2 + sqrt(2)*beta*|10>/2

Apply Hadamard Q_10
Q_10=sqrt(2)*alpha*|00>/2 + sqrt(2)*alpha*|10>/2 - sqrt(2)*beta*|01>/2 + sqrt(2)*beta*|11>/2
c=sqrt(2)*alpha*|00>/2 + sqrt(2)*alpha*|10>/2 - sqrt(2)*beta*|01>/2 + sqrt(2)*beta*|11>/2

Apply Hadamard Q_11
Q_11=sqrt(2)*alpha*|01>/2 - sqrt(2)*alpha*|11>/2 - sqrt(